In [2]:
import tkinter as tk
from tkinter import Label, Button
from PIL import Image, ImageTk
import json
from pathlib import Path
import platform  # 운영 체제 감지용

# 현재 운영 체제 확인
is_windows = platform.system() == "Windows"

# 분류할 이미지 리스트
work_dir_path = Path("W:\wine\wine_recognition\WORK_dataset_collection\data\dynamic\work_assignment\Labeler1_한명훈")

img_path_list = list((work_dir_path / "1st/sam_masked_image").rglob("*.jpg"))
img_path_list = sorted(img_path_list, key=lambda x: int(x.parent.parent.name[2:]) * 10000 + int(x.parent.name[1]) * 100 + int(x.stem))
img_path_list = [path.relative_to(work_dir_path) for path in img_path_list ]
img_path_list = [str(path).replace("\\", "/") for path in img_path_list]


# JSON 저장 파일
output_json = work_dir_path / "1st/wrong_image_path.json"

# 기존 데이터 로드
if output_json.exists():
    with open(output_json, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
            if not isinstance(data, dict):  # 데이터가 딕셔너리가 아니면 초기화
                raise ValueError("JSON 데이터가 올바른 형식이 아닙니다.")
        except Exception:
            data = {img: {"checked": False, "error": False} for img in img_path_list}
else:
    data = {img: {"checked": False, "error": False} for img in img_path_list}

current_index = next((i for i, img in enumerate(img_path_list) if not data.get(img, {"checked": False, "error": False})["checked"]), 0)

img_tk = None  # 글로벌 이미지 객체 유지

# 초기 이미지 크기 설정
img_width, img_height = 500, 500  # 기본 크기
min_size, max_size = 200, 1000  # 최소/최대 크기 제한

# GUI 창 생성
root = tk.Tk()
root.title("Image Classification")

# 이미지 번호 표시 라벨
image_number_label = Label(root, text="", font=("Arial", 12))  # 이미지 번호 표시용
image_number_label.pack()

# 이미지 표시 라벨
img_label = Label(root)
img_label.pack()

# 체크 상태 라벨
status_label = Label(root, text="Unchecked", font=("Arial", 14))
status_label.pack()

def resize_image(event):
    """CTRL 키와 마우스 휠로 이미지 크기 조정 (Windows & Ubuntu 지원)"""
    global img_width, img_height
    if event.state & 0x4:  # CTRL 키가 눌린 상태인지 확인
        if is_windows:
            delta = event.delta
        else:  # Ubuntu (Linux)인 경우
            delta = 4 if event.num == 4 else -4
        
        if delta > 0:  # 스크롤 업(확대)
            img_width = min(img_width + 50, max_size)
            img_height = min(img_height + 50, max_size)
        else:  # 스크롤 다운(축소)
            img_width = max(img_width - 50, min_size)
            img_height = max(img_height - 50, min_size)
        load_image()  # 이미지 다시 로드하여 크기 적용

def load_image():
    """ 현재 인덱스의 이미지를 로드하여 표시 """
    global current_index, img_tk  # 전역 변수 유지
    if not img_path_list:
        status_label.config(text="No images left to classify.")
        return
    
    img_path = img_path_list[current_index]
    
    if img_path not in data:
        data[img_path] = {"checked": False, "error": False}

    data[img_path]["checked"] = True

    try:
        image = Image.open(work_dir_path/img_path)
        image = image.resize((img_width, img_height), Image.Resampling.LANCZOS)  # 동적 크기 적용
        img_tk = ImageTk.PhotoImage(image)
        img_label.config(image=img_tk)
        img_label.image = img_tk  # 객체 유지

        # 이미지 번호 업데이트
        image_number_label.config(text=str(Path(*Path(img_path).parts[-3:]))+f"\n{current_index + 1} / {len(img_path_list)}")

        # 상태 업데이트
        status_label.config(text=f"Checked: {'✔' if data[img_path]['checked'] else '❌'} | "
                                 f"Error: {'Incorrect' if data[img_path]['error'] else 'Correct'}")
    except Exception as e:
        print(f"⚠️ 이미지 로드 실패: {img_path} - {e}")
        status_label.config(text="⚠️ 이미지 로드 실패")

def next_image():
    """ 다음 이미지로 이동 """
    global current_index
    if current_index < len(img_path_list) - 1:
        current_index += 1
        load_image()
    save_and_exit()

def prev_image():
    """ 이전 이미지로 이동 """
    global current_index
    if current_index > 0:
        current_index -= 1
        load_image()
    save_and_exit()

def toggle_error():
    """ 현재 이미지의 오류 상태 토글 """
    img_path = img_path_list[current_index]
    if img_path in data:
        data[img_path]["error"] = not data[img_path]["error"]
        data[img_path]["checked"] = True  # 확인한 이미지로 설정
        status_label.config(text=f"Checked: {'✔' if data[img_path]['checked'] else '❌'} | "
                                 f"Error: {'Incorrect' if data[img_path]['error'] else 'Correct'}")
    save_and_exit()

def save_and_exit():
    """ JSON 파일로 체크 상태 저장 후 종료 """
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

# 키보드 이벤트 바인딩
root.bind("<Right>", lambda event: next_image())  # → 키 다음
root.bind("<Left>", lambda event: prev_image())   # ← 키 이전
root.bind("<space>", lambda event: toggle_error())  # 스페이스 키 오류 토글

# 운영 체제에 따라 마우스 휠 이벤트 바인딩
if is_windows:
    root.bind("<MouseWheel>", resize_image)  # Windows
else:
    root.bind("<Button-4>", resize_image)  # Ubuntu 스크롤 업
    root.bind("<Button-5>", resize_image)  # Ubuntu 스크롤 다운

# 종료 버튼
exit_button = Button(root, text="Save & Exit", command=save_and_exit)
exit_button.pack()

# GUI 실행 전에 100ms 후 `load_image()` 실행
root.after(100, load_image)
root.mainloop()


'w:\\wine\\sam2\\notebooks'